<a href="https://colab.research.google.com/github/rashmi0888/Kaggle/blob/master/Digit%20Recognizer/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digit Recognizer for MNIST dataset

### Import relevant components

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
np.random.seed(100)
# Disable warnings
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)

### Data Preparation
#### 1. Load Data
Read data from csv file.

In [0]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

#### 2. Data Dimensions
Dataset consists of 42,000 training and 28,000 test images. Each image is a 28x28 greyscale image having pixel values in range 0-255. Each training image has its corresponding label, a number in range 0-9, which represents the image. Labels for test images have to be predicted.

In [0]:
training_samples = 42000
test_samples = 28000
input_dim = 784
num_output_classes = 10

In [6]:
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### 3. Reshape and Normalize Data
1. Separate pixel values and labels. 
2. Convert pixel values from 0-255 to 0-1. 
3. Convert labels into one hot vectors.

In [0]:
X_train, y_train = df.iloc[:, 1:], df.iloc[:, 0]
X_test = df_test

X_train /= 255.0
X_test /= 255.0

Y_train = pd.get_dummies(y_train)

#### 4. Shuffle Data
   Randomly shuffle data to later get a good validation set.

In [0]:
random_index = np.random.permutation(X_train.shape[0])
X_train, Y_train, y_train = X_train.iloc[random_index], Y_train.iloc[random_index], y_train.iloc[random_index]

### Train Model
#### 1. Build model
1. Build a DNN configuration of 3 hidden layers having 512 units each.
2. Sequential model is a linear stack of layers. On this model, add 4 dense layers which are fully connected layers. 
3. 3 dense layers correspond to 3 hidden layers and 4th one is output layer.
4. First layer needs info about input shape. Subsequent layers can do automatic shape inference.

#### 2. Compile model
1. After building model, configure the learning process.
2. Momentum parameter is used in learning.

#### 3. Fit model
1. Fit the model built above into the data.
2. Out of 42,000 traning samples, use 7,000 samples for validation.

#### 4. Tune parameters
1. Train and evaluate the model for various combinations of learning rates and batch sizes to choose best one.

In [0]:
def train_model(learning_rate, batch_size, epochs):
    # Build model
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(input_dim,)))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(num_output_classes, activation='softmax'))
    
    # Compile model
    sgd = SGD(lr=learning_rate, momentum=0.9, decay=1e-6)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Fit model
    history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, verbose=0, validation_split=0.16666,
                        shuffle=True)
    print('Train Accuracy:{}, Validation Accuracy:{}'.format(history.history['acc'][-1], 
                                                             history.history['val_acc'][-1])) 
    
    # Make predictions
    digits = model.predict_classes(X_test, verbose=0)
    
    return digits

In [29]:
# Tune parameters
learning_rate = [0.001, 0.01, 0.05, 0.1]
batch_size = [32, 128, 256, 1024]
grid = list(itertools.product(learning_rate, batch_size))
for params in grid:
    print("\nLearning_rate: {}, Batch_size: {}".format(params[0], params[1]))
    train_model(params[0], params[1], epochs = 50)


Learning_rate: 0.001, Batch_size: 32
Train Accuracy:0.9999428571428571, Validation Accuracy:0.9758571427890232

Learning_rate: 0.001, Batch_size: 128
Train Accuracy:0.984200000013624, Validation Accuracy:0.9672857145581927

Learning_rate: 0.001, Batch_size: 256
Train Accuracy:0.9651714286395482, Validation Accuracy:0.9539999999318804

Learning_rate: 0.001, Batch_size: 1024
Train Accuracy:0.9218857142175947, Validation Accuracy:0.9211428542137146

Learning_rate: 0.01, Batch_size: 32
Train Accuracy:1.0, Validation Accuracy:0.9797142857142858

Learning_rate: 0.01, Batch_size: 128
Train Accuracy:0.9999714285714286, Validation Accuracy:0.9754285714285714

Learning_rate: 0.01, Batch_size: 256
Train Accuracy:1.0, Validation Accuracy:0.974857142993382

Learning_rate: 0.01, Batch_size: 1024
Train Accuracy:0.9888571428843907, Validation Accuracy:0.968714284147535

Learning_rate: 0.05, Batch_size: 32
Train Accuracy:1.0, Validation Accuracy:0.9822857142857143

Learning_rate: 0.05, Batch_size: 128

### Make Predictions
Use best parameters of learning rate = 0.1 and batch size = 128 to make predictions on test set.

In [0]:
digits = train_model(learning_rate=0.1, batch_size=128, epochs=50)
df_final = pd.DataFrame({'ImageId':df_test.index+1, 'Label':digits})
df_final.to_csv('Prediction.csv', index = False)

In [0]:
## Scored 0.98114